In [3]:
from stable_baselines3 import PPO,

from agents.DQN_agent import DoubleDQNAgent # for typing only

from environment.env import GYMGridEnvDeform, FULLGYMGridEnvDeform

In [4]:
import numpy as np
import itertools
import matplotlib.pyplot as plt

from eval.eval import eval_agent, all_data
# maze size
N = 2

# thetas deformations (range(a,b),range(c,d))
l0 = 1
h0 = 10
l1 = 1
h1 = 10

maze = np.load(f"maze/maze_{N}.npy")
env = FULLGYMGridEnvDeform(maze,l0,h0,l1,h1, render_mode="human")

states = [((x,y,phi),(i,j)) for x in range(1,env.max_shape[0]-1) for y in range(1,env.max_shape[1]-1) for phi in range(4) for i in range(l0,h0) for j in range(l1,h1)] 
positions = [(x,y,phi) for x in range(1,env.max_shape[0]-1) for y in range(1,env.max_shape[1]-1) for phi in range(4)]
actions = [0,1,2,3]
obs = list(itertools.product([0,1], repeat=5))
thetas = [(i,j) for i in range(l0,h0) for j in range(l1,h1)]

state_dict = {state: i for i, state in enumerate(states)}
position_dict = {position: i for i, position in enumerate(positions)}
obs_dict = {obs : i for i, obs in enumerate(obs)}

# Actions are: 0-listen, 1-open-left, 2-open-right
lenS = len(states)
lenP = len(positions)
lenA = len(actions)
lenO = len(obs)


# PPO Train

In [4]:
from stable_baselines3.common.callbacks import BaseCallback
from wandb.integration.sb3 import WandbCallback
import wandb

class My_callback(BaseCallback):
    def __init__(self, verbose=0):
        super(My_callback, self).__init__(verbose)
    def _on_step(self) -> bool:
        if self.num_timesteps % 200 == 0:
            self.training_env.reset()
        return True
    def _on_rollout_end(self) -> None:
        print(f"Rollout end: {self.num_timesteps}")
        return True
    

total_timesteps = 100000
batch_size = 2000
n_steps = 2000

config = {
    "policy_type": "MultiInputPolicy",
    "env_name": "FULLGYMGridEnvDeform",
    "defo_range": (l0,h0,l1,h1),
    "maze_size": N,
    "total_timesteps": total_timesteps,
    "Batch_Size": batch_size,
    "PPO n_steps": n_steps
}

run = wandb.init(
    project="PPO",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)

callbacks = [My_callback(0), 
             WandbCallback(gradient_save_freq=100,
                            model_save_path=f"models/{run.id}",
                            verbose=2,
                            ),
            ]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: matteo-nunziante (adv_topics). Use `wandb login --relogin` to force relogin


In [5]:
total_timesteps = 100000
batch_size = 2000
n_steps = 2000


In [ ]:
# n_steps (int) – The number of steps to run for each environment per update 
# (i.e. rollout buffer size is n_steps * n_envs
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder

def make_env():
    N = 2

    # thetas deformations (range(a,b),range(c,d))
    l0 = 1
    h0 = 10
    l1 = 1
    h1 = 10
    
    maze = np.load(f"maze/maze_{N}.npy")
    env = FULLGYMGridEnvDeform(maze,l0,h0,l1,h1, render_mode="human")

    env = Monitor(env)  # record stats such as returns
    return env

env = DummyVecEnv([make_env])


model = PPO("MultiInputPolicy",env,n_steps=n_steps,batch_size=batch_size,verbose=1,tensorboard_log=f"runs/{run.id}", device="cpu")

model.load("models/PPO_3s3uf88g.zip


NameError: name 'run' is not defined

In [ ]:
model.learn(total_timesteps,progress_bar=True, callback=callbacks)
model.save(f"models/PPO_{run.id}")
env.close()
run.finish()


In [ ]:
model.save("ppo_MDP_1")

# PPO eval

In [8]:
total_timesteps = 100000
batch_size = 2000
n_steps = 2000


In [12]:

model = PPO("MultiInputPolicy",env,n_steps=n_steps,batch_size=batch_size,verbose=1, device="cpu")

model.load("agents/pretrained/PPO_a2svroc5.zip")

max_steps = 200

env = FULLGYMGridEnvDeform(maze,l0,h0,l1,h1,render_mode="human")
obs, _ = env.reset()
done = False
steps = 0
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env.step(action)
    steps += 1
    if done or steps > max_steps:
        obs, _ = env.reset()
        steps = 0
        done = False


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(


error: display Surface quit

# DQN

In [2]:
from stable_baselines3 import DQN

model = DQN("MlpPolicy",env,verbose=1, tensorboard_log="runs/DQN", device="cpu")


NameError: name 'env' is not defined

## value function MDP

In [ ]:
state_value = pass


SyntaxError: invalid syntax (908321876.py, line 1)

: 

In [ ]:

# fix orientation and deformation
deformation = (2, 1)

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

for orientation in range(4):
    Value_matrix_plot = np.zeros(env.maze.shape) - np.inf
    for s, state in enumerate(states):
        if state[1] == deformation and state[0][2] == orientation:
            Value_matrix_plot[state[0][0], state[0][1]] = state_value[s]
    
    ax = axes[orientation]
    ax.imshow(Value_matrix_plot)
    ax.set_title(f"Orientation: {orientation}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

plt.suptitle("Value Function Matrices for Different Orientations")
plt.tight_layout()
plt.show()